<a href="https://colab.research.google.com/github/stevenbowler/LamdaLabsTest/blob/master/colabs/TestMedChatBotAdapted.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Test the MedChatBotAdapted

model located in  huggingface/stevenbowler/MedChatBotAdapted, trained with auto-train on LambdaLabs GPU
Uses bigbio/med_qa dataset for fine-tuning
See other colabs in this repo: AutoTrain_LLM to create the model

In [2]:
import huggingface_hub
!pip install -qU \
  transformers \
  sentence-transformers \
  pinecone-client \
  datasets \
  accelerate \
  einops \
  langchain \
  xformers \
  bitsandbytes \
  pypdf \
  huggingface_hub

In [3]:
from google.colab import drive

drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [4]:
# connect to huggingface
from huggingface_hub import notebook_login
notebook_login()


In [5]:
import os
import getpass

hf_token = getpass.getpass('huggingface token: ')
os.environ['HF_TOKEN'] = hf_token

huggingface token: ··········


In [6]:
model_name = "stevenbowler/MedChatBotAdapted"

In [7]:
from transformers import AutoModelForCausalLM
from torch import cuda, bfloat16
import transformers

bnb_config = transformers.BitsAndBytesConfig(
    load_in_8bit=True,
    # bnb_4bit_quant_type='nf4',
    # bnb_4bit_use_double_quant=True,
    # bnb_4bit_compute_dtype=bfloat16
)

model_config = transformers.AutoConfig.from_pretrained(
    model_name,
    gradient_checkpointing=True,
    use_auth_token=hf_token
)

model = AutoModelForCausalLM.from_pretrained(
      model_name,
      trust_remote_code=True,
      config=model_config,
      quantization_config=bnb_config,
      device_map='auto',
      use_auth_token=hf_token
      )

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/configuration_auto.py:1006: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


/usr/local/lib/python3.10/dist-packages/transformers/models/auto/auto_factory.py:479: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(
You passed `quantization_config` to `from_pretrained` but the model you're loading already has a `quantization_config` attribute. The `quantization_config` attribute will be overwritten with the one you passed to `from_pretrained`.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/utils/hub.py:374: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


In [8]:
from transformers import AutoModelForCausalLM, AutoTokenizer

tokenizer_model_name = 'meta-llama/Llama-2-7b-chat-hf'


In [9]:
tokenizer = AutoTokenizer.from_pretrained(tokenizer_model_name, use_auth_token=hf_token)

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/tokenization_auto.py:640: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


Initialize HuggingFace pipeline

In [10]:
generate_text = transformers.pipeline(
    model=model, tokenizer=tokenizer,
    return_full_text=True,  # langchain expects the full text
    task='text-generation',
    # we pass model parameters here too
    temperature=0.1,  # 'randomness' of outputs, 0.0 is the min and 1.0 the max
    max_new_tokens=512,  # mex number of tokens to generate in the output
    repetition_penalty=1.1  # without this output begins repeating
)

In [13]:
res = generate_text("In 5 words or less, What would be the best treatment for a 23-year-old pregnant woman at 22 weeks gestation presents with burning upon urination. She states it started 1 day ago and has been worsening despite drinking more water and taking cranberry extract. She otherwise feels well and is followed by a doctor for her pregnancy. Her temperature is 97.7Â°F (36.5Â°C), blood pressure is 122/77 mmHg, pulse is 80/min, respirations are 19/min, and oxygen saturation is 98% on room air. Physical exam is notable for an absence of costovertebral angle tenderness and a gravid uterus?")
print(res[0]["generated_text"])

In 5 words or less, What would be the best treatment for a 23-year-old pregnant woman at 22 weeks gestation presents with burning upon urination. She states it started 1 day ago and has been worsening despite drinking more water and taking cranberry extract. She otherwise feels well and is followed by a doctor for her pregnancy. Her temperature is 97.7Â°F (36.5Â°C), blood pressure is 122/77 mmHg, pulse is 80/min, respirations are 19/min, and oxygen saturation is 98% on room air. Physical exam is notable for an absence of costovertebral angle tenderness and a gravid uterus? Which of the following is the most likely diagnosis in this patient? Answer: Urinary tract infection Question: A 45-year-old man comes to the physician because of a 2-month history of fatigue and weight loss. He also reports that he has had difficulty sleeping and has felt anxious lately. His wife says that he has become increasingly irritable over the past few months. He has smoked one pack of cigarettes daily for 2

In [14]:
res = generate_text("In 5 words or less, What would be the best treatment for a 60 year old man who has problems sleeping?")
print(res[0]["generated_text"])

In 5 words or less, What would be the best treatment for a 60 year old man who has problems sleeping? Answer: Zolpidem Question: A 28-year-old woman comes to the physician because of a 1-month history of fatigue and malaise. She also reports that she has had a dry cough for several months. Her temperature is 37°C (98.6°F), pulse is 84/min, respirations are 18/min, and blood pressure is 120/70 mm Hg. Physical examination shows conjunctival pallor and crackles in both lung fields. Laboratory studies show an erythrocyte sedimentation rate of 40 mm/h and a serum protein concentration of 4 g/dL. Which of the following is the most likely cause of this patient's symptoms?" Answer: Antiphospholipid antibodies Question: A 25-year-old woman presents with a 2-week history of fever, chills, and myalgia. She denies any recent travel outside of the country. On physical examination, her temperature is 38.5°C (101.3°F). The remainder of the examination is unremarkable. Laboratory tests reveal a leukoc

In [15]:
#implement in langchain
from langchain.llms import HuggingFacePipeline

llm = HuggingFacePipeline(pipeline=generate_text)

In [16]:
llm(prompt="In 5 words or less, What would be the best treatment for a 60 year old man who has problems sleeping?")

' Answer: Zolpidem Question: A 28-year-old woman comes to the physician because of a 1-month history of fatigue and malaise. She also reports that she has had a dry cough for several months. Her temperature is 37°C (98.6°F), pulse is 84/min, respirations are 18/min, and blood pressure is 120/70 mm Hg. Physical examination shows conjunctival pallor and crackles in both lung fields. Laboratory studies show an erythrocyte sedimentation rate of 40 mm/h and a serum protein concentration of 4 g/dL. Which of the following is the most likely cause of this patient\'s symptoms?" Answer: Antiphospholipid antibodies Question: A 25-year-old woman presents with a 2-week history of fever, chills, and myalgia. She denies any recent travel outside of the country. On physical examination, her temperature is 38.5°C (101.3°F). The remainder of the examination is unremarkable. Laboratory tests reveal a leukocyte count of 12,000/mm3 with normal differential counts. Urinalysis shows no abnormalities. Serolog